In [2]:
import tensorflow as tf

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)

print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


nie wypisał, bo trzeba <i>zainicjować</i>

In [4]:
import tensorflow as tf

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)

print(node1, node2)
sess = tf.Session()
print(sess.run([node1, node2]))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)
[3.0, 4.0]


Teraz dodamy węzeł dodający

In [5]:
import tensorflow as tf
from __future__ import print_function

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)

print(node1, node2)
sess = tf.Session()

node3 = tf.add(node1, node2)
print("node3: ", node3)
print("nowa sesja:", sess.run(node3))

Tensor("Const_2:0", shape=(), dtype=float32) Tensor("Const_3:0", shape=(), dtype=float32)
node3:  Tensor("Add:0", shape=(), dtype=float32)
nowa sesja: 7.0


Czy musimy inicjować na nowo zmienione wierzchołki?

In [7]:
import tensorflow as tf
from __future__ import print_function

node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0)

print(node1, node2)
sess = tf.Session()

node2 = tf.constant(5.0)

node3 = tf.add(node1, node2)
print("node3: ", node3)
print("nowa sesja:", sess.run(node3))

Tensor("Const_7:0", shape=(), dtype=float32) Tensor("Const_8:0", shape=(), dtype=float32)
node3:  Tensor("Add_2:0", shape=(), dtype=float32)
nowa sesja: 8.0


Nie musimy - sess run działa rekurencyjnie?

# dodaj wierzchołki nie będące stałymi

In [18]:
import tensorflow as tf
from __future__ import print_function

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

adder = a + b

#karmimy addera wartościami poszczegolnych wierzcholkow

print(sess.run(adder, {a: 3, b: 4.5}))
print(sess.run(adder, {a: [1, 3], b: [2, 4]}))

7.5
[ 3.  7.]


tak możemy manipulować wejściem do tensora (dodawania)

In [19]:
import tensorflow as tf
from __future__ import print_function

a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)

adder = a + b
tf.identity(adder, "testZmiany") #zmieniamy wartosc po fakcie


#karmimy addera wartościami poszczegolnych wierzcholkow

print(sess.run(adder, {a: 3, b: 4.5}))
print(sess.run(adder, {a: [1, 3], b: [2, 4]}))

tripler = adder * 3

print("multi3:", sess.run(tripler, {a: 3, b: 4.5}))

7.5
[ 3.  7.]
multi3: 22.5


# budujemy model do nauki

In [23]:
import tensorflow as tf
from __future__ import print_function

W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32)

direction = tf.multiply(W, x, name='kierunek')
linear = tf.add(direction, b, name='linear')

init = tf.global_variables_initializer()
sess.run(init) 
# trzeba puscic zeby zainicjalizowac zmienne do nauki 

print(sess.run(linear, {x:[1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


teraz dodajemy prawidłowe y (placeholder) i funkcję liczącą odstępstwo od normy

In [26]:
y = tf.placeholder(tf.float32) #specjalny typ danych?
squaredDeltas = tf.square(linear - y, name='kwadratOdejmowania') #rozumie, ze linear to wynik? NIE - to TENSOR <3

loss = tf.reduce_sum(squaredDeltas, name='liczIleBrakuje')
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


inicjalizowanie zmiennej - powazna sprawa
zmiana to 'assign', po niej trzeba zrobic sess run

In [27]:
fixW = tf.assign(W, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))
'''
#poprawiamy
refixW = tf.assign(W, [.3])
refixb = tf.assign(b, [-.3])
sess.run([refixW, refixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))'''

0.0


'\n#poprawiamy\nrefixW = tf.assign(W, [.3])\nrefixb = tf.assign(b, [-.3])\nsess.run([refixW, refixb])\nprint(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))'

## trenowanie 

In [28]:
optimizer = tf.train.GradientDescentOptimizer(0.01, name="Optimizer")
train = optimizer.minimize(loss, name="Trainer")


sess.run(init) #przywraca wejciowe inicjalizacje

for i in range(1000):
    sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})
    
print(sess.run([W, b]))

writer = tf.summary.FileWriter("./logs", sess.graph)

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


# wytrenowane

# tensorBoard

In [29]:
writer = tf.summary.FileWriter("./logs", sess.graph)